In [6]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.tensorboard.plugins.projector as tf_projector
import scipy.io.arff
from numpy.lib.recfunctions import append_fields
from numpy.lib.recfunctions import drop_fields
from time import gmtime, strftime, time
from sklearn.model_selection import train_test_split
import sklearn as skl
import random
import os
from sklearn.preprocessing import OneHotEncoder
np.set_printoptions(threshold=np.nan)

In [7]:
"""
Este tutorial cobre a utilização da biblioteca para o diagnóstico de Doença Renal Crónica com uma rede neuronal do tipo
feed-forward, visualização de métricas de treino e finalmente visualização dos dados usando Tensorboard. 
O dataset usado, assim como a descrição de cada atributo e classe está disponível em 
https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease

Tensorflow é uma biblioteca de computação numérica baseada em grafos de computação. Construimos o grafo adicionando-lhe nós
que representam operações matemáticas. Estas operam sobre tensores, que são a generalização de escalares, vectores, matrizes,
etc a qualquer número de dimensões e constituem o tipo de dados principal da biblioteca. No entanto estas apenas correm
ao chamar a função run - a avaliação é lazy.
"""

one = tf.constant(1)                            # tf usado para aceder à library tensor constante

x = tf.placeholder(                             # placeholder é um valor que será fornecido ao correr o grafo
    dtype="int32",                              # tipo do placeholder
    shape=[],                                   # forma [] representa um escalar
    name="x"                                    # nome do tensor
) 
c = tf.add(one, x)                              # retorna tensor com a soma de one e x, que ainda não tem valor

with tf.Session() as sess:                      # sessões encapsulam ambiente em que correm as operações
    res = sess.run(c, feed_dict={x:2})          # calcular o valor de c, substituindo x pelo valor 2, na sessão sess
    print(res)

3


In [8]:
a = tf.constant(1)
b = tf.constant(2)
c = tf.constant(3)
d = tf.constant(4)

ab = tf.add(a,b)
bc = tf.add(b,c)
cd = tf.add(c,d)

with tf.Session() as sess:
    a_vl, ab_val, bc_val = sess.run([a, ab, bc]) # apenas são avaliadas as operações de que dependem as operações 
                                                 # passadas em run(), neste caso, cd não é avaliado. 
                                                 # run() retorna um valor por cada operação que lhe é passada
    print(a_vl, ab_val, bc_val)


1 3 5


In [25]:
"""
O primeiro passo consiste no pré-processamento dos dados. O dataset contém 400 individuos, cada um com 24 atributos e uma 
classificação binária, com ou sem doença. Alguns destes atributos são nominais e terão de ser convertidos para numéricos.
Cada atributo nominal com n valores possíveis é substituído por n atributos binários, que indicam se o indíviduo tem ou não
aquele valor no atributo original. Os atributos binários não necessitam desta conversão mas esta é feita neste caso para 
simplificar o tratamento dos dados.

Inicialmente é feita a conversão dos campos nominais para vários binários, depois são substituídos os valores desconhecidos
e finalmente, os dados são normalizados

"""

filename = "chronic_kidney_disease.arff"
data, meta = scipy.io.arff.loadarff(filename)
data_copy = np.copy(data)

"""
lb = skl.preprocessing.LabelBinarizer()
for name, ty in meta.__dict__.get("_attributes").items():
    print(name, ty)
    if ty[0] == "nominal":
        lb.fit([st.encode(encoding='UTF-8') for st in ty[1]])
        temp = lb.transform(data[name]).astype(np.float64)
        print(temp)
"""

def pre_process(data, class_name):
    # contagem do número de classes existentes para serem separados dos atributos posteriormente
    num_classes = 0
    # vai permitir substituir os valores dum nparray por floats
    # se o valor for desconhecido, é substituído por NaN, se o valor
    # for igual ao passado, é substituido por 1, se não, é substituído por 0
    vec = lambda preset: np.vectorize(lambda v: np.nan if v == b'?' else 1. if v == preset else 0.)
    for i, field in enumerate(data.dtype.names):
        type_of_field = meta.types()[i]                       # tipo do campo pode ser nominal ou numérico
        
        if type_of_field == "nominal":                        # apenas serão tratados os campos nominais
            
            valid = data[field][data[field] != b'?']          # extrair os valores conhecidos
            
            unique = np.unique(valid).tolist()                # lista com os valores possíveis do atributo atual
            
            for val in unique:
                # para cada valor val possível do atributo field, criamos um novo campo chamado field_val
                # que terá o valor 1 para os indivíduos com o valor val no atributo field e 0 nos restantes
                new_field = field + "_" + val.decode("utf-8") 
                data = append_fields(
                    data,
                    new_field,
                    vec(val)(data[field])
                )
            data = drop_fields(data, field)                   # eliminar campo original
            
    for i, field in enumerate(data.dtype.names):
        if field[:len(class_name)] == class_name:             # não normalizar classes
            num_classes += 1
            continue
        valid = data[field][~np.isnan(data[field])]           # extrair valores conhecidos
        
        std = valid.std()                                     
        mean = valid.mean()
        
        data[field][np.isnan(data[field])] = mean             # subsituir valores desconhecidos pela média dos valores 
                                                              # conhecidos do campo atual
            
        data[field] = (data[field] - mean) / std              # normalizar valores, subtraindo a média e divindo pelo
                                                              # desvio padrão do campo atual, resulta numa distribuição 
                                                              # com média 0 e desvio padrão unitário
    return num_classes, data

num_classes, data = pre_process(data=data, class_name="class")

In [48]:
"""
Construção da rede

"""

# construção de uma camada da rede com  num_units neurónios
# a utilização de keyword arguments é opcional e é usada aqui para explicitar os argumentos usados
def make_layer(inputs, num_units, name, final = True):
    with tf.name_scope(name):                          # name scope permite agrupar as operações da mesma camada
        weights = tf.Variable(                         # variáveis têm trainable=True por defeito;
            initial_value= tf.truncated_normal(        # pesos inicializados aleatóriamente com uma distribuição normal
                [inputs.shape[1].value, num_units],    # inicialização duma matriz de tamanho nr colunas de input * nº de neurónios
                dtype=tf.float64                       # tipo dos valores gerados
            ),                     
            name=name + "_weights"                     # nome permite identificar fácilmente a variável no Tensorboard
        )
        biases = tf.Variable(
            tf.zeros(                                  # inicializar a 0s
                [num_units],                           # um valor por neurónio
                dtype=tf.float64
            ),    
            name=name + "_biases"
        )
        #permite a visualização da distribuição das variáveis ao longo do treino
        tf.summary.histogram(
            name="weights", 
            values=weights
        ) 
        tf.summary.histogram(
            name="biases",
            values=biases
        )
        
        # multiplicação de matrizes
        ret = tf.matmul(inputs, weights) + biases      # soma com tensores equivalente a tf.add()
        
        if not final:
            # função de ativação ReLU https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
            # não é usada na camada final
            ret = tf.nn.relu(features=ret)
            
    return ret

def make_net(inputs, hidden_units, classifications):
    in_val = inputs
    for i, num_units in enumerate(hidden_units):
        # criar uma camada intermédia para cada elemento de hidden_units
        in_val = make_layer(
            in_val, 
            num_units, 
            "layer_{}".format(i),
            False
        )
    # camada final
    out_val = make_layer(
        in_val, 
        classifications.shape[1].value, 
        "final_layer",  
        True
    )
    
    with tf.name_scope("post"): # cálculo de probabilidades, custo, 
        # softmax transforma outputs da camada final nas probabilidades para cada classe
        # https://en.wikipedia.org/wiki/Softmax_function
        probabilities = tf.nn.softmax(
            logits=out_val, 
            name="soft_max"
        )
        
        cost = tf.reduce_mean(                           # custo é média da cross entropy para todas as classes
            tf.nn.softmax_cross_entropy_with_logits(     # http://neuralnetworksanddeeplearning.com/chap3.html
                labels=classifications,                  # classificações vindas dos dataset
                logits=out_val,                          # valores de saída da camada final
                name="cross"
            ), 
            name="cost"
        )
        
        # optimizer permite treinar os pesos e biases da rede
        # ao avaliar esta operação, o gradiente é calculado automáticamente
        # e propagado pela rede, atualizando o valor dos pesos e biases
        # existem vários otimizadores definidos em tf.train
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=0.01,                          # usamos uma learning rate constante, mas pode variar com o step
            name="opt"
        ).minimize(                                      # indicar que o custo deve ser minimizado
            cost, 
            name="minimizer"
        )           
        
        # a rede escolhe a classificação com um maior valor de output da camada final
        # classificação é o índice do maior valor
        predictions = tf.argmax(
            input=out_val,
            dimension=1,
            name="predictions"
        )
        
        # são correctas as previsões iguais às labels do dataset
        correct_predictions = tf.equal(
            x=predictions,
            y=tf.argmax(
                input=classifications,                   # vindos do data set
                dimension=1
            ), 
            name="correct"
        )
        
        # precisão é a fracção de previsões feitas corretamente
        # idealmente deve ser calculada para cada classe
        accuracy = tf.reduce_mean(
            tf.cast(                                     # conversão de booleanos para floats para calcular a média
                x=correct_predictions, 
                dtype=tf.float64
            ), 
            name="acc"
        )
        
        # permite a visualização das métricas de custo e precisão ao longo do treino
        tf.summary.scalar(
            name="acc", 
            tensor=accuracy
        )
        tf.summary.scalar(
            name="cost", 
            tensor=cost
        )
    
    return probabilities, optimizer, predictions, correct_predictions, accuracy
        

In [49]:
# converter o structured array para um array numpy de floats
data_array = data.view(np.float64)

# mudar a forma do array para ter número de linhas correspondente ao número de indíviduos
data_array = data_array.reshape(data.shape[0], -1)

# extrair apenas os atributos
# para todas as linhas, extrair da primeira coluna até a última - num_classes coluna
inputs = data_array[:, :-num_classes]

# extrair as classes
# para todas as linhas, extrair as últimas num_classes colunas
labels = data_array[:, -num_classes:]

In [50]:
# fazer reset ao grafo, para eliminar nós já introduzidos
tf.reset_default_graph()
                                                         #
# vai conter os atributos dos invíduos
X = tf.placeholder(                                  
    dtype="float64",                                     # tipo de dados do tensor
    
    shape=[None, inputs.shape[1]],                       # o tensor passado será uma matriz com uma linha por indivíduo
                                                         # e uma coluna por atributo. O número de indíviduos pode variar
                                                         # e por isso é usado o valor None para este
    name="X"
)

# vai conter as classificações vindas do dataset
y = tf.placeholder(
    dtype="float64", 
    shape=[None, labels.shape[1]], 
    name="y"
)

keep_prob = tf.placeholder(
    dtype=tf.float64, 
    name="keep_prob"
)

# chamada da função de construção da rede definida acima
probabilities, optimizer, predictions, correct_predictions, accuracy = make_net(X, [], y)

# operação de inicialização das variávies (pesos e biases)
init = tf.global_variables_initializer()

# número máximo de iterações
num_epochs = 1000
error = 0.001

# percentagem dos indíviduos a usar no conjunto de teste
# é usado para verificar o poder de previsão da rede em indíviduos
# que não foram usados no treino
test_size = 0.5

# percentagem do indíviduos do conjunto de treino a usar no conjunto de validação
validation_size = 0.2

# operação que junta todos as operações de summary
merged = tf.summary.merge_all()

random.seed(42)
seeds = random.sample(range(100000), num_epochs)

LOG_DIR = os.path.join('.', 'logs')

def make_file_writer(dataset, curr_time, session):
    return tf.summary.FileWriter(
        os.path.join(LOG_DIR, "{}", "{}").format(dataset, curr_time),
        session.graph
    )

# baralha os indivíduos e separa-os em conjunto de treino e teste
# stratified significa que a divisão é feita mantendo a mesma proporção
# de indivíduos de cada classe em cada 
kf = skl.model_selection.StratifiedShuffleSplit(
    n_splits=1, 
    test_size=test_size, 
    random_state=random.randrange(10000)
)


# iterar sobre as diferentes formas de separar os dados em treino e teste
# como n_splits=1 acima, apenas é retornada uma
# os dados são separados uma única vez em treino e teste
# em cada iteração, os dados de treino vão ser separados em treino e validação
for train_temp_i, test_i in kf.split(inputs, labels):   
    
    # split retorna os indices a usar, aqui obtemos os dados que os indices referem
    train_temp_inputs = inputs[train_temp_i]
    train_temp_labels = labels[train_temp_i]
    test_inputs       = inputs[test_i]
    test_labels       = labels[test_i]
    
    with tf.Session() as sess:
        # usamos o tempo atual para por os dados de cada run num directório diferente
        curr_time = time()
        
        # para escrever estatísticas em ficheiros ao longo do processo de treino
        train_writer = make_file_writer("train", curr_time, sess)
        valid_writer = make_file_writer("valid", curr_time, sess)
        test_writer  = make_file_writer("test" , curr_time, sess)
        
        sess.run(init)
        for i in range(num_epochs):
            # em cada iteração, separar os dados de treino nos conjuntos de validação e de treino
            # mantendo proporção de indíviduos de cada classe
            kf2 = skl.model_selection.StratifiedShuffleSplit(n_splits=1, test_size=validation_size, random_state=seeds[i])
            
            for train_i, validation_i in kf2.split(train_temp_inputs, train_temp_labels):
                # aceder aos indices retornados
                train_inputs      = train_temp_inputs[train_i]  
                train_labels      = train_temp_labels[train_i]
                validation_inputs = train_temp_inputs[validation_i]
                validation_labels = train_temp_labels[validation_i]
                
                # treino da rede
                _, stats = sess.run(
                    [optimizer, merged],
                    feed_dict={X: train_inputs, y: train_labels}
                )
                
                # escrever as estatísticas de treino para o passo i
                train_writer.add_summary(
                    summary=stats,
                    global_step=i
                )        
                
                # validação (como não é passada a operação de minimização, a rede não é treinada)
                acc, stats = sess.run(
                    [accuracy, merged], 
                    feed_dict={X: validation_inputs, y: validation_labels}
                )
                
                # escrever as estatísticas de validação para o passo i
                valid_writer.add_summary(
                    summary=stats,
                    global_step=i
                )        
            
            # periodicamente verificar a performance da rede no conjunto de treino
            # a rede nunca é treinada com este conjunto
            if i % 100 == 0:
                # ao não passar a operação de minimização, a rede não é treinada
                acc, stats = sess.run(
                    [accuracy, merged], 
                    feed_dict={X: test_inputs, y: test_labels}
                )
                # escrever as estatísticas de teste para o passo i
                test_writer.add_summary(
                    summary=stats, 
                    global_step=i
                )         
                print("Test Accuracy: ", acc, i)    

Test Accuracy:  0.305 0
Test Accuracy:  0.73 100
Test Accuracy:  0.785 200
Test Accuracy:  0.815 300
Test Accuracy:  0.85 400
Test Accuracy:  0.86 500
Test Accuracy:  0.875 600
Test Accuracy:  0.89 700
Test Accuracy:  0.895 800
Test Accuracy:  0.905 900


In [38]:
path_for_metadata =  os.path.join(LOG_DIR,'metadata.tsv')

embedding_var = tf.Variable(inputs, name="attributes")
summary_writer = tf.summary.FileWriter(LOG_DIR)

config = tf_projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# escrever metadados num ficheiro .tsv (tab separated values)
with open(path_for_metadata,'w') as f:
    sep = "\t"
    f.write("Index")
    for field in data.dtype.names:
        f.write(sep + field)
    f.write(sep + "Labels\n")
    for index in range(len(inputs)):
        f.write("{}".format(index))
        for field_i, field in enumerate(data.dtype.names):
            f.write("{}{}".format(sep, data[field][index] + random.uniform(0, 0.0001)))
            
        f.write("{}{}\n".format(sep, 'ckd' if labels[index][0] == 1 else 'notckd'))
        
embedding.metadata_path = path_for_metadata #'metadata.tsv'


# visualizar embeddings
tf_projector.visualize_embeddings(summary_writer, config)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt2"), 1)

'.\\minimalsample\\model.ckpt2-1'

In [41]:
os.path.join(".", "logs", "{}").format(1)

'.\\logs\\1'

str